In [10]:
import openai
import os
import re
import requests
from dotenv import load_dotenv
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain.schema import HumanMessage, AIMessage
from langchain.llms import AzureOpenAI
from langchain.chains import ConversationalRetrievalChain
from typing import Tuple, List, Dict, Callable
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter, Document
from langchain.embeddings import HuggingFaceEmbeddings


load_dotenv()
API_KEY = os.getenv("OPENAI_API_KEY") 
openai.api_type = "azure"
openai.api_base = "https://open-ai-maspex.openai.azure.com"
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")


url = openai.api_base + "/openai/deployments?api-version=2022-12-01"

r = requests.get(url, headers={"api-key": API_KEY})
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')


In [11]:
load_dotenv()
file_path = './data/book3.pdf'
loader = PyPDFLoader(file_path)
text = loader.load()

In [12]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(text)
db = FAISS.from_documents(docs, embeddings)
vector_store = db.save_local('faiss_index_karny')

In [13]:
# docs = db.similarity_search("What are B-splines", k=2)
# for doc in docs:
#     print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

In [14]:
new_db = FAISS.load_local('faiss_index_karny', embeddings)

In [19]:
def make_chain():
    model = AzureOpenAI(
        deployment_name="chat",
        model_name='gpt-35-turbo'
    )

    embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

    vector_store = FAISS.load_local('faiss_index_karny', embeddings)

    return ConversationalRetrievalChain.from_llm(
        model,
        retriever=vector_store.as_retriever(),
        return_source_documents=True
    )


if __name__ == '__main__':
    load_dotenv()

    chain = make_chain()
    chat_history = []

    while True:
        print()
        question = input("Enter a question: ")

        response = chain({"question": question, "chat_history": chat_history})

        answer = response["answer"]
        source = response["source_documents"]
        chat_history.append(HumanMessage(content=question))
        chat_history.append(AIMessage(content=answer))

        print("/n/nSources:\n")
        for document in source:
            print(f"Page: {document.metadata['page']}")
            print(f"Text: {document.page_content}...\n")
        print(f"Answer: {answer}")
    


/n/nSources:

Page: 58
Text: ©Kancelaria Sejmu   s. 59/164 
 
2023 -05-17 
 § 4a. W razie skazania na grzywnę zatarc ie skazania następuje z  mocy prawa 
z upływem roku od wykonania lub darowania kary albo od przedawnienia jej 
wykonania.  
§ 5. W razie odstąpienia od wymierzenia kary, zatarcie skazania następuje 
z mocy prawa z  upływem roku od wydania prawomocnego orzeczenia.  
§ 6. Jeżeli orzeczono środek karny, przepadek lub środek kompensacyjny, 
zatarcie skazania nie może nastąpić przed jego wykonaniem, darowaniem albo 
przedawnieniem jego wykonania. Zatarcie skazania nie może nastąpić również 
przed wykonaniem środka zabezpiec zającego.  
Art. 107a.  (uchylony)  
Art. 108. Jeżeli sprawcę skazano za dwa lub więcej nie pozostających 
w zbiegu przestępstw, jak również jeżeli skazany po rozpoczęciu, lecz przed 
upływem, okresu wymaganego do zatarcia skazania ponownie popełnił 
przestępstwo,  dopuszczalne jest tylko jednoczesne zatarcie wszystkich skazań.  
Rozdział XIII  
Odpowiedz

KeyboardInterrupt: Interrupted by user

In [ ]:
document.metadata

{'source': './data/book2.pdf', 'page': 312}